## 1. MyRNNcell 클래스 구현하기


In [3]:
import torch
import torch.nn as nn

class MyRNNcell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MyRNNcell, self).__init__()
        self.hidden_size = hidden_size

        self.Wx = nn.Parameter(torch.randn(hidden_size, input_size))
        self.Wh = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b = nn.Parameter(torch.randn(hidden_size))

## 확인하기
model_rnn = MyRNNcell(input_size=4, hidden_size=6)
model_rnn

MyRNNcell()

방금 'init' 함수만으로 생성한 SimpleRNN모델의 내부 파라미터를 출력해봅니다.

In [5]:
for name, param in model_rnn.named_parameters():
  print(f"{name}: {param.size()}")

Wx: torch.Size([6, 4])
Wh: torch.Size([6, 6])
b: torch.Size([6])


클래스 정의하기: forward함수 구현

In [7]:
import torch
import torch.nn as nn

# SimpleRNN 클래스 정의
class MyRNNcell(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(MyRNNcell, self).__init__()
    self.hidden_size = hidden_size
    self.Wx = nn.Parameter(torch.randn(hidden_size, input_size))
    self.Wh = nn.Parameter(torch.randn(hidden_size, hidden_size))
    self.b  = nn.Parameter(torch.randn(hidden_size))

  def forward(self, x, hidden):
    # .t()는 전치 행렬
    hidden = torch.tanh(torch.matmul(x,self.Wx.t()) + torch.matmul(hidden, self.Wh.t()) + self.b)
    return hidden

# 모델 인스턴스 생성
model_rnn = MyRNNcell(input_size =4, hidden_size= 6)
model_rnn

MyRNNcell()

MyRNNcell 인스턴스를 통한 새로운 은닉 상태 계산

In [8]:
torch.manual_seed(42)

# 랜덤 입력 데이터 생성
x_t = torch.randn(1,4)

# 초기 은닉 상태 생성
initial_hidden = torch.randn(1,6) # hidden_size = 6

# 모델 실행
new_h_t = model_rnn(x_t, initial_hidden)
print("새로운 은닉 상태", new_h_t)

새로운 은닉 상태 tensor([[ 0.9969,  0.2076,  0.9999, -0.3315, -0.8195,  0.9997]],
       grad_fn=<TanhBackward0>)


시퀀스 데이터 처리를 위한 MyRNN 클래스 구현

앞에서 정의한 MyRNNcell 클래스를 사용하여 RNN cell의 동작을 정의하고, 이를 MyRNN 클래스에 내장하여 전체 시퀀스를 처리하는 MyRNN 클래스를 정의합니다.

In [9]:
import torch
import torch.nn as nn

class MyRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MyRNN, self).__init__()
    self.hidden_size = hidden_size
    self.rnn_cell = MyRNNcell(input_size, hidden_size)
    self.Wy = nn.Parameter(torch.randn(output_size, hidden_size))

model_myrnn = MyRNN(input_size = 4, hidden_size = 6, output_size=2)
model_myrnn

MyRNN(
  (rnn_cell): MyRNNcell()
)

init_hidden 함수 내부 구현

In [13]:
import torch
import torch.nn as nn

class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_cell = MyRNNcell(input_size, hidden_size)
        self.Wy = nn.Parameter(torch.randn(output_size, hidden_size))

    def init_hidden(self, batch_size=1):
        return torch.zeros(batch_size, self.hidden_size)

model_myrnn = MyRNN( input_size =4, hidden_size = 6, output_size=2)
h0 = model_myrnn.init_hidden()
h0

tensor([[0., 0., 0., 0., 0., 0.]])

forward 함수 구현


In [15]:
import torch
import torch.nn as nn

class MyRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MyRNN, self).__init__()
    self.hidden_size = hidden_size
    self.rnn_cell = MyRNNcell(input_size, hidden_size)
    self.Wy = nn.Parameter(torch.randn(output_size, hidden_size))

  def init_hidden(self, batch_size = 1):
    return torch.zeros(batch_size, self.hidden_size)

  def forward(self, x):
    h_t = self.init_hidden(x.size(0))

    for i in range(x.size(1)):
      h_t = self.rnn_cell(x[:,1], h_t)

    output = torch.sigmoid(torch.matmul(h_t, self.Wy.t()))
    return output, h_t

# 모델 초기화을 통한 인스턴스 생성 예시
model_myrnn = MyRNN(input_size = 4, hidden_size=6, output_size = 2)
model_myrnn


MyRNN(
  (rnn_cell): MyRNNcell()
)

Sequence 입력 데이터와 MyRNN을 통한출력 계산

In [16]:
sequence_data = torch.tensor([
    [0.1, 0.2, 0.3, 0.4],
    [0.2, 0.3, 0.4, 0.5],
    [0.3, 0.4, 0.5, 0.6],
    [0.4, 0.5, 0.6, 0.7],
    [0.5, 0.6, 0.7, 0.8],
])

# MyRNN 모델 인스턴스화
input_size = 4  # 입력 피처의 개수
hidden_size = 6  # 은닉 상태의 크기
output_size = 1  # 출력 크기

model_myrnn1 = MyRNN(input_size, hidden_size, output_size)

# 모델을 통해 시퀀스 데이터 처리
output, ht = model_myrnn1(sequence_data.unsqueeze(0))
print("입력 시퀀스:", sequence_data)
print(f"모델 출력:, {output} {ht}")

입력 시퀀스: tensor([[0.1000, 0.2000, 0.3000, 0.4000],
        [0.2000, 0.3000, 0.4000, 0.5000],
        [0.3000, 0.4000, 0.5000, 0.6000],
        [0.4000, 0.5000, 0.6000, 0.7000],
        [0.5000, 0.6000, 0.7000, 0.8000]])
모델 출력:, tensor([[0.2572]], grad_fn=<SigmoidBackward0>) tensor([[ 0.8949, -0.9999,  0.9107, -0.9973, -0.9982, -0.9982]],
       grad_fn=<TanhBackward0>)
